# npZigZagPlotter
Draws zigzag lines over stock price charts. Finds turning points. 

User controls
1. Stock ticker list. Add, delete tickers. Enclose each ticker with single quotes, followed by a comma, as shown.
2. Set PIVOT_POINT_PCT, which is written as a decimal fraction. This is the minimum price movement that will cause a pivot point. For instance, if PIVOT_POINT_PCT = 0.025, only price movements >= 2.5% will cause a pivot point. 
3. DAYS_BACK = Number of trading days to include in the chart. 250 trading days per year. 

# TODO
Developer ToDo list 04/17/23.
0. Broker the time inputs to Darqube. Fix 'em.
1. Set col 'time' as index.
2. Save price data.
3. Check price data to request only prices that are not already downloaded.

In [1]:
# **********
#
#ZigZagPlotter - created 04/06/23
#    Goal - Plot zigzag price trends overlaid on stock candlestick charts.
#
#     Steps
#          1. Initialize libraries, constants, etc.
#          2. Get prices for selected tickers from DARqube.
#          3. Calculate turning  ponts price_change delta for date D from date D - MAX_SPIKE DAYS
#
# **********

# **********
#
# User sets constants
#
# **********

# Set DAR_key
DAR_key = '90180f15ecc74513a01ca017eca2bb4f'
# Set TIKCERS = ('XXX', 'YYY')
TICKERS = ('DDOG', 'SNOW', 'CRWD') #, 'FSLY', 'OKTA', 'NWBO', 'SANA', 'ONCS', 'BBAI')

# Set minimum pivot point size as a percent. Adjust this to produce a chart where each zigzag takes 5-25 trading days.
PIVOT_POINT_PCT = .05

# Set DAYS_BACK as an integer number of days back from yesterday.
DAYS_BACK = 250 # approx. 1 year. Note DAYS_BACK uses trading days, not calendar days.

# **********
#
# Import LIbraries
#
# **********

import csv
import copy
import json
import matplotlib.pyplot as plt
from   matplotlib.pyplot import plot, scatter

import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import stock_indicators
from   stock_indicators import indicators, EndType
from   stock_indicators.indicators.common.enums import EndType

import time
import yfinance as yf
#import npzigzag
#from npzigzag import core as zz
#import zigzag

import datetime
from   datetime import datetime
from   datetime import date
#from   datetime import fromtimestamp
from   dateutil.relativedelta import relativedelta



In [2]:
# **********
#
#ZigZagPlotter - functions that cal the ZigZag pivot points.
#
# **********

def pct_change(X):
    data_pct_change = diff(X, 1, None) / shift(X,1, None) 
    return data_pct_change

def shift(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

def diff(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[num:] - arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[:num] - arr[-num:]
    else:
        result[:] = arr
    return result

def calczigzag(X, pc, include_first):

    data_pct_change = pct_change(X)
    pct_change_mask = np.sign(data_pct_change)
    pct_change_mask_abs_diff = np.abs(diff(pct_change_mask,1,np.nan))
    split_mask = np.where(pct_change_mask_abs_diff == 2)[0] - 1
    
    data_split_pct_change = pct_change(X[split_mask])

    data_split_pct_change_filtered_indices = np.where(np.abs(data_split_pct_change) > pc)
    data_split_pct_change_filtered = data_split_pct_change[data_split_pct_change_filtered_indices]
    pivot_indices = split_mask[data_split_pct_change_filtered_indices]
    pivot_indices_filtered = pivot_indices[diff(np.sign(data_split_pct_change_filtered),-1,None)!= 0]
    if include_first:
        pivot_indices_filtered = np.concatenate(([0],pivot_indices_filtered))
        
    return pivot_indices_filtered

def zigzag(X, pc, include_first = True):
    '''
    X: numpy.ndarray/list/pandas.core.series.Series
        Data
    pc: float
        Precision level
    include_first: bool
        Boolean indicating whether to include the first observation as a pivot point
    '''

    if type(X) is np.ndarray:
        pivot_indices = calczigzag(X, pc, include_first)
        return pivot_indices

    elif type(X) is pd.Series:
        X_np = X.values 
        X_index = X.index
        pivot_indices = calczigzag(X_np, pc, include_first)
        return X_index[pivot_indices]

    elif type(X) is list:
        X_np = np.array(X)
        pivot_indices = calczigzag(X_np, pc, include_first)
        return pivot_indices
    
    else:
        raise ValueError("X should be pd.series, np.array or list")

print('Loaded the zigzag functions.')


Loaded the zigzag functions.


In [3]:
# **********
#
# Main Program
# 1. Get stock prices
# 2. Calculate pivot points.
# 3. Plot for each ticker.
#
# **********

# My debug attempts.
#this_date_float = 1680048000
#this_date_int   = 1680048000
#this_date = datetime.date.fromtimestamp(this_date_float)
#print(this_date_float, ' = ', str(this_date_int))
#print('this_date date = ', this_date_float.strftime('%Y-%m-%d'))
     
ticker_prices = pd.DataFrame(columns = ['time', 'Ticker', 'price'])

#new_months = my_datetime - relativedelta(months = 4)
TODAY = datetime.today()
print('TODAY = ', TODAY)
ts = TODAY
print(ts)
#print(datetime.timestamp(date_only))

end_date = datetime.timestamp(TODAY - relativedelta(days = 1))
print('end_date = ', end_date)
end_date_str = str(int(end_date))
print('end_date_str = ', end_date_str)
start_date = datetime.timestamp(TODAY - relativedelta(days = DAYS_BACK))
print('start_date = ', start_date)
start_date_str = str(int(start_date))
print('start_date_str = ', start_date_str)
for ticker in TICKERS:
    
    ticker_df = pd.DataFrame()
    price_rqst = 'https://api.darqube.com/data-api/market_data/historical/' + ticker + '?token=' + DAR_key + \
                    '&start_date=' + start_date_str + '&end_date=' + end_date_str + '&interval=1d'
    
    #price_rqst = 'https://api.darqube.com/data-api/market_data/quote/' + ticker + '?token=' + DAR_key
    response = requests.get(price_rqst)
    price_dict = response.json()
    ticker_prices = pd.DataFrame(data = price_dict)
    print()
    print('ticker_prices direct from DARqube')
    print()
    # print(ticker_prices.head(6))
    print()
    print(ticker_prices['time'].head(8))
    print()
    ticker_prices['float_time'] = ticker_prices['time'].astype(float)
    print()
    ticker_prices['date'] = pd.to_datetime(ticker_prices['float_time'])
    print('ticker_prices['date'])
    print(ticker_prices.date.head(10))
    
    stop
    
    ticker_prices.set_index('time', drop = False, append = False, inplace = True)
    print(ticker_prices.head(7))
    ticker_prices['timestamp'] = pd.to_datetime(ticker_prices['time'], unit='ms')
    print(ticker_prices.head(7))
    X = ticker_prices['adjusted_close']
    zz_pivots = zigzag(X,PIVOT_POINT_PCT, False)
    
    stop
    print('for ', ticker)
    plot(X, '--')
    plot(zz_pivots, X[zz_pivots])
    scatter(zz_pivots, X[zz_pivots])
    plt.show()
    print(ticker, ' plot done.')
    print()
    print()
    
print('thats all folks')


TODAY =  2023-04-17 14:07:25.569869
2023-04-17 14:07:25.569869
end_date =  1681679245.569869
end_date_str =  1681679245
start_date =  1660165645.569869
start_date_str =  1660165645

ticker_prices direct from DARqube


0    1660176000
1    1660262400
2    1660521600
3    1660608000
4    1660694400
5    1660780800
6    1660867200
7    1661126400
Name: time, dtype: int64


ticker_prices.date = 
0   1970-01-01 00:00:01.660176000
1   1970-01-01 00:00:01.660262400
2   1970-01-01 00:00:01.660521600
3   1970-01-01 00:00:01.660608000
4   1970-01-01 00:00:01.660694400
5   1970-01-01 00:00:01.660780800
6   1970-01-01 00:00:01.660867200
7   1970-01-01 00:00:01.661126400
8   1970-01-01 00:00:01.661212800
9   1970-01-01 00:00:01.661299200
Name: date, dtype: datetime64[ns]


NameError: name 'stop' is not defined